In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
import numpy as np
from torchvision import datasets, transforms, models

data_dir = '/floyd/input/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])
#data_transforms = transforms.Compose([transforms.Resize(255),
#                                transforms.CenterCrop(224),
#                                transforms.ToTensor()])
#dataset = datasets.ImageFolder(data_dir, transform=train_transforms)
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)


In [2]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [3]:
# Loading network
model = models.vgg16(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# move the model to GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:06<00:00, 90599770.24it/s]


cuda


In [4]:
#defining a classifier
classifier = nn.Sequential(nn.Linear(7 * 7 * 512, 4096),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(4096, 2024),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(2024, 102),
                          nn.LogSoftmax(dim=1))

model.classifier = classifier

In [5]:
# *** Specify Loss Function and Optimizer ***
import torch.optim as optim

criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.002)


In [6]:
# *** Train the Network ***
# number of epochs to train the model
model.to(device)
n_epochs = 5
valid_loss_min = np.Inf # track change in validation loss
for epoch in range(1, n_epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    # train the model #
    model.train()
    for data, target in trainloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        logprobs = model.forward(data)
        # calculate the batch loss
        loss = criterion(logprobs, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()
    # validate the model #
    model.eval()
    with torch.no_grad():
        for data, target in validloader:
            # move tensors to GPU if CUDA is available
            data, target = data.to(device), target.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            logprobs = model.forward(data)
            # calculate the batch loss
            loss = criterion(logprobs, target)
            # update average validation loss
            valid_loss += loss.item()
    # calculate average losses
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    # print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'models/model_flowers.pt')
        valid_loss_min = valid_loss
# // *** Train the Network ***

Epoch: 1 	Training Loss: 0.062930 	Validation Loss: 0.042601
Validation loss decreased (inf --> 0.042601).  Saving model ...
Epoch: 2 	Training Loss: 0.033633 	Validation Loss: 0.042101
Validation loss decreased (0.042601 --> 0.042101).  Saving model ...
Epoch: 3 	Training Loss: 0.029775 	Validation Loss: 0.032892
Validation loss decreased (0.042101 --> 0.032892).  Saving model ...
Epoch: 4 	Training Loss: 0.028149 	Validation Loss: 0.037028
Epoch: 5 	Training Loss: 0.026540 	Validation Loss: 0.032706
Validation loss decreased (0.032892 --> 0.032706).  Saving model ...


In [8]:
checkpoint = {'epoch':n_epochs,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict':optimizer.state_dict(),
               'loss':loss}

torch.save(checkpoint, 'models/checkpoint.pth')

In [12]:
def prepare_model():
    model = models.vgg16(pretrained=True)
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
    #defining a classifier
    classifier = nn.Sequential(nn.Linear(7 * 7 * 512, 4096),
                              nn.ReLU(),
                              nn.Dropout(0.2),
                              nn.Linear(4096, 2024),
                              nn.ReLU(),
                              nn.Dropout(0.2),
                              nn.Linear(2024, 102),
                              nn.LogSoftmax(dim=1))
    model.classifier = classifier
    return model

def load_checkpoint(filepath):
    model = prepare_model()
    model.load_state_dict(checkpoint['model_state_dict'])
    
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.002)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return model

model_new = load_checkpoint('models/checkpoint.pth')

In [13]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    image_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
    return image_transforms(image)

In [14]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [63]:
from PIL import Image
import os

def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {i: classes[i] for i in range(len(classes))}
    return class_to_idx

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    pil_img = Image.open(img_path)
    image = process_image(pil_img)
    image.unsqueeze_(0)
    model.eval()
    logps = model.forward(image)
    ps = torch.exp(logps)
    top_p, top_class = ps.topk(5, dim=1)
    top_p = np.squeeze(top_p.detach().numpy())
    top_p = [val for val in top_p]
    top_class = np.squeeze(top_class.detach().numpy())
    idx_to_classes = find_classes(valid_dir)
    top_class_keys = [idx_to_classes[value] for value in top_class]
    return top_p, top_class_keys 

img_path = valid_dir + '/56/image_02771.jpg'
print(predict(img_path, model_new))
actual_class = cat_to_name[img_path.split('/')[5]]
print("Actual class: {}".format(actual_class))
#labels = {int(key):value for (key, value) in cat_to_name}
#os.listdir(valid_dir)
#classes = [d.name for d in os.scandir(valid_dir) if d.is_dir()]
#print(classes)
#imshow(image)

([0.9999943, 6.1996348e-06, 6.723769e-08, 2.5966582e-08, 2.6282478e-09], ['56', '59', '46', '58', '100'])
Actual class: bishop of llandaff
